In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline

import numpy as np
import imageio
import visvis as vv
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os 
import cv2
import torch
from PIL import Image as im
from PIL import Image  
from PIL import Image as im
import numpy as np

from skimage.metrics import structural_similarity as ssim
import argparse
import shutil
import os

import art
from art.estimators.classification import BlackBoxClassifier
from art.defences.preprocessor import JpegCompression
from art.attacks.evasion import HopSkipJump
from art.attacks.evasion import ZooAttack
from art.utils import to_categorical
from art.utils import load_dataset, get_file, compute_accuracy

In [ ]:
import cv2
import imageio
import time
from matplotlib import pyplot as plt
from PIL import Image  
from PIL import Image as im
import numpy as np

from skimage.metrics import structural_similarity as ssim
import argparse
import shutil
import os



base_images=[]
folders=[]
folders_full=[]
save_folders=[]
csv_saves=[]
folder_4_base=r"E:\selected dataset\bases for all category" #folder containing base images of each target class
folder=r"E:\selected dataset\MIT withoutbase" #folder containing multiple classes folder of target images

save_folder=r"E:\selected dataset\demo save" 


for filename in os.listdir(folder_4_base):
        base_images.append((os.path.join(folder_4_base,filename)))
        
for filename in os.listdir(folder):
        folders.append((os.path.join(folder,filename)))
        
for filename in os.listdir(save_folder):
        save_folders.append(os.path.join(save_folder,filename))

N=25 #total number of boxes
M=12 #picked number of boxes
    

for main_loop in range(10): #number of classes 

    img2_arr=[]
    img3_arr=[]
    imgO_arr=[]
    imgnew_arr=[]
    savee=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        savee.append('\p'+str(i)+'.jpg')

    img1_arr=imageio.imread(base_images[main_loop])
    a=0
    b=0

    for filename in os.listdir(folders[main_loop]):
            img2_arr.append(imageio.imread(os.path.join(folders[main_loop],filename)))
            imgO_arr.append(imageio.imread(os.path.join(folders[main_loop],filename)))
    v=0
 ########################################################################################################
    for i in range(len(img2_arr)):
        
        image_init=[]
        image_target=[]
        averaged=[]
        

        windowsize_r = 80
        windowsize_c = 80

        a=0
        for r in range(0,img1_arr.shape[0], windowsize_r):
            for c in range(0,img1_arr.shape[1], windowsize_c):
                arr=img1_arr[r:r+windowsize_r,c:c+windowsize_c,:]     
                image=im.fromarray(arr)
                image_init.append(np.array(image))
        for r in range(0,img2_arr[i].shape[0], windowsize_r):
            for c in range(0,img2_arr[i].shape[1], windowsize_c):
                arr=img2_arr[i][r:r+windowsize_r,c:c+windowsize_c,:]     
                image=im.fromarray(arr)
                image_target.append(np.array(image))
                
                
        sift = cv2.xfeatures2d.SURF_create()
        def predict(x):
    
            out_label = []
            global dst_pts1,dst_pts2,src_pts1,src_pts2
            for x_i in x:

              x_i = x_i / x_i.max() #normalizes data in range 0 - 255
              x_i = 255 * x_i
              x_i = x_i.astype(np.uint8)

              method = eval('cv2.TM_CCOEFF_NORMED')
              res = cv2.matchTemplate(image_init1,x_i,method)
              min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)


              if max_val < 0.7:
                out_label.append(0)
              elif max_val >= 0.9:
                out_label.append(1)
              else:
                out_label.append(2)


            return to_categorical(out_label, 3)

        # init black box object
        classifier = BlackBoxClassifier(predict, image_target[0].shape, 3, clip_values=(0, 255))
        label_dict = {0: 'image2', 1: 'image1', 2: 'other'}
        attack = HopSkipJump(classifier=classifier, targeted=True, norm=2, max_iter=0, max_eval=100, init_eval=10,verbose=False)
        iter_step = 10
        a=0
        list1=[]
        merge=[]
        for i in range(1,129,1):
            list1.append('image'+str(i)+'.jpg')

        for j in range(25):
            image_target1=image_target[j]
            image_init1=image_init[j]
            plt.imshow(image_init[j])
            plt.show()
            for i in range(10):
                x_adv = attack.generate(x=np.array([image_target[j]], dtype=np.float32), x_adv_init=np.array([image_init[j]],dtype=np.float32), y=to_categorical([1], 3))
                print("Adversarial image at step %d." % (i * iter_step), "L2 error", 
                      np.linalg.norm(np.reshape(x_adv[0] - image_target[j], [-1])),
                      "and RANSAC output %s." % label_dict[np.argmax(classifier.predict(x_adv)[0])])

                plt.imshow(x_adv[0]/255)
                plt.show(block=False)
                print(x_adv[0].shape)
                xtemp=x_adv[0]
                x_adl=xtemp.astype(np.uint8)
                arr2im=im.fromarray(x_adl)
                merge.append(arr2im)
                arr2im.save(list1[a])
                
                attack.max_iter = iter_step
            a=a+1


        from PIL import Image
        len(merge)
        image1=[]
        blank_image = Image.new("RGB", (400, 400))
        for a in range(25):
            print(a)
            image1.append(Image.open('image'+str(a)+'.jpg'))



        list1=[0,80,160,240,320]
        k=0
        for i in list1:
            for j in list1:
                blank_image.paste(image1[k], (j,i))
                k=k+1




  ##########################################################################################################          
        im2=blank_image
        im3=blank_image
      
        im2 = im2.save(save_folders[main_loop]+savee[v]) 
        imgnew_arr.append(imageio.imread(save_folders[main_loop]+savee[v]))
        v=v+1
    def calculate_SSIM(imageA,imageB):
        s = ssim(imageA, imageB,multichannel=True)
        
        return s
    SSIM1=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        SSIM1.append(calculate_SSIM((imgO_arr[i]),(imgnew_arr[i])))
    print("SSIM between target & adversary image",SSIM1)
    def calculate_PSNR(im1,im2):
        P=cv2.PSNR(im1,im2)
        
        return P
    PSNR1=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        PSNR1.append(calculate_PSNR(imgO_arr[i],imgnew_arr[i]))
    print("PSNR between target & adversary image",PSNR1)


    def feature_match(check):

        sift = cv2.xfeatures2d.SURF_create()
        kpNEW1, desNEW1 = sift.detectAndCompute(check, None)
        kpNEW2, desNEW2 = sift.detectAndCompute(adv_image, None)

        pts_adv0 = cv2.KeyPoint_convert(kpNEW1)
        pts_init0 = cv2.KeyPoint_convert(kpNEW2)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        matchesgood1 = flann.knnMatch(desNEW1,desNEW2,k=2)


        # store all the good matches as per Lowe's ratio test.
        good1 = []
        for m,n in matchesgood1:
            if m.distance < 0.7*n.distance:
                good1.append(m)

        src_pts1 = np.float32([ kpNEW1[m.queryIdx].pt for m in good1 ]).reshape(-1,1,2)
        dst_pts1 = np.float32([ kpNEW2[m.trainIdx].pt for m in good1 ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts1, dst_pts1, cv2.RANSAC,3.0)
        mask=list(mask)
        inliers=mask.count(1)
       
        return inliers
    adv_image=img1_arr
    inliers_OP=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        inliers_OP.append(feature_match(imgnew_arr[i]))
    adv_image=img1_arr
    inliers_OO=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        inliers_OO.append(feature_match(imgO_arr[i]))
    print("feature match before attack",inliers_OO)
    print("feature match after attack",inliers_OP)
    PFMI=[]
    for i in range(len(os.listdir(folders[main_loop]))):
        try:
            PFMI.append(str(((inliers_OP[i]-inliers_OO[i])/inliers_OO[i])*100)+str('%'))
        except:
            PFMI.append(inliers_OP[i])
    print("percentage feature matching increase",PFMI)
